In [3]:
import torch

In [9]:
from torch.serialization import add_safe_globals
from torch.utils.data import TensorDataset

add_safe_globals([TensorDataset])

train_dataset = torch.load('../data/processed/train_dataset.pt', weights_only=False)
test_dataset = torch.load('../data/processed/test_dataset.pt', weights_only=False)


In [10]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
from train_model import BasicCNN, get_resnet18_transfer_model

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Instantiate both models
model_basic = BasicCNN().to(device)
model_resnet18 = get_resnet18_transfer_model().to(device)
